<h1>Creating Custom Datasets and DataLoaders in PyTorch for Training</h1>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torchvision.io import read_image
from torch.utils.data import Dataset
import os
from PIL import Image
import torch.nn.functional as F
from pathlib import Path

In [2]:
class CustomImageDataset(Dataset):
    def __init__(self, root_dir, split, transform=None):
        """
        Args:
            root_dir (string): Directory with all the images.
            split (string): 'train' or 'test' to specify the dataset split.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.root_dir = root_dir
        self.split = split
        self.transform = transform
        self.img_labels = []

        split_dir = os.path.join(root_dir, split)
        categories = ['cats', 'dogs']
        for category in categories:
            category_dir = os.path.join(split_dir, category)
            for img_name in os.listdir(category_dir):
                img_path = os.path.join(category_dir, img_name)
                label = 0 if category == 'cats' else 1
                self.img_labels.append((img_path, label))

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path, label = self.img_labels[idx]
        image = Image.open(img_path).convert('RGB') 
        #image = read_image(img_path).float() / 255.0  # Normalize to [0,1]
        if self.transform:
            image = self.transform(image)
        return image, label

# Example usage:
# dataset = CustomImageDataset(root_dir='path/to/dataset', split='train', transform=ToTensor())
# train_loader = DataLoader(dataset, batch_size=4, shuffle=True)


In [3]:
# Define the image transformations
transform = transforms.Compose([
    transforms.Resize((64, 64)),  # Resize to match the input size of the model
    transforms.ToTensor(),
])

Dataset Link :- https://www.kaggle.com/datasets/samuelcortinhas/cats-and-dogs-image-classification?resource=download
(cats and Dog image classification)

In [5]:
# Load the custom dataset

Dataset_path = Path('C:/Users/amitc/Downloads/archive_1/')  # dir path

# traning
train_dataset = CustomImageDataset(root_dir=Dataset_path, split='train', transform=transform) 
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# testing
test_dataset = CustomImageDataset(root_dir=Dataset_path, split='test', transform=transform)
test_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=False)

In [6]:
# Define a simple image classification model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(16 * 32 * 32, 2)  # Adjust based on the number of classes

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = x.view(-1, 16 * 32 * 32)  # Flatten the tensor
        x = self.fc1(x)
        return x

In [7]:
# Initialize the model, loss function, and optimizer
model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [8]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_dataloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 10 == 9:  # Print every 10 batches
            print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 10:.3f}')
            running_loss = 0.0

print('Finished Training')

[Epoch 1, Batch 10] loss: 1.013
[Epoch 2, Batch 10] loss: 0.666
[Epoch 3, Batch 10] loss: 0.652
[Epoch 4, Batch 10] loss: 0.627
[Epoch 5, Batch 10] loss: 0.620
[Epoch 6, Batch 10] loss: 0.595
[Epoch 7, Batch 10] loss: 0.559
[Epoch 8, Batch 10] loss: 0.515
[Epoch 9, Batch 10] loss: 0.516
[Epoch 10, Batch 10] loss: 0.467
Finished Training


In [10]:
# Move the model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Evaluate the model
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_dataloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10 test images: {100 * correct / total:.2f}%')

Accuracy of the network on the 10 test images: 86.36%
